## PROJECT TITLE: CUSTOMER TRANSACTIONS

#### Data Analysis by: Kwabena Boateng

#### IMPORT LIBRARY PACKAGES

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from datetime import datetime

#from sklearn.preprocessing import LabelEncoder 

#### LOAD DATASET

In [ ]:
customer_transactions = pd.read_csv("C:/Users/KWABENABOATENG/Desktop/DATA ANALYSIS/PERSONAL/CUSTOMER TRANSACTIONS PROJECT/sample_dataset.csv")

customer_transactions

#### DATA UNDERSTANDING

The dataset used for this project is stored in the data/ directory. The data consists of customer transaction records with the following columns:

* customer_id: Unique identifier for each customer.
* Name: First name of the customer.
* Surname: Last name of the customer.
* Gender: the sexual orientation of the customer.
* Birthdate: the day, month and year the customer was born.
* Transaction Amount: Amount of transaction made.
* Date: Date of the transaction.
* Merchant Name: the name of the merchant making the transactions.
* Category: Product category.

#### DATA PREPARATION

###### CHECK NUMBER OF COLUMNS AND ROWS IN THE DATASET.

In [ ]:
print('The number of rows and columns in this dataset is', customer_transactions.shape, 'respectively')

#### CHECK THE STRUCTURE OF THE DATASET

In [ ]:
#Show all the names of the columns in this dataset.

customer_transactions.columns

In [ ]:
#Rename columns title.
customer_transactions = customer_transactions.rename(columns={'Name': 'First Name', 'Surname': 'Last Name', 
                                                              'Birthdate': 'Date of Birth', 
                                                              'Transaction Amount': 'Transaction Amt ($)'})

In [ ]:
#top first (5)records of the dataset.

customer_transactions.head()

In [ ]:
customer_transactions.info()

#### CHECK FOR MISSING VALUES

In [ ]:
customer_transactions.isna().sum()

In [ ]:
Total_MV = customer_transactions.isna().sum().sum()
print('The total number of missing values in this dataset is', Total_MV)

In [ ]:
#Check for the percentage for the missing values.

missing_gender = customer_transactions['Gender'].isnull().sum()
total_rows = len(customer_transactions)
missing_percentage = (missing_gender / total_rows) * 100

print(f"Missing Gender Values: {missing_gender} ({missing_percentage:.2f}%)")

#### INSIGHTS

The results showed that there are 5,047 missing values in the Gender column of the dataset. The percentage of the missing value sums up to 10.09% which is quite a huge percentage.


In dealing with this, I will replace all the missing values in the gender column to UNDEFINED.

This will presuppose that the sexual and gender orientation was not available for them to select, hence they left that field empty.

#### DEALING WITH THE MISSING VALUES IN THE GENDER COLUMN

In [ ]:
#Checking for the unique values in the Gender column.

customer_transactions['Gender'].unique()

In [ ]:
#Checking for the count of the unique values in the Gender column.

customer_transactions['Gender'].value_counts()

In [ ]:
#Replace missing values in the gender column with undefined.

customer_transactions['Gender'].fillna('Undefined', inplace=True)

In [ ]:
#Replace gender initials with the full word.

customer_transactions['Gender'] = customer_transactions['Gender'].replace('M', 'Male')
customer_transactions['Gender'] = customer_transactions['Gender'].replace('F', 'Female')

In [ ]:
#Checking for the unique values in the Gender column after replacing the missing values and renaming the initials with full word.
customer_transactions['Gender'].unique()

#### CHECK THE CATEGORY COLUMNS

In [ ]:
customer_transactions['Category'].unique()

In [ ]:
customer_transactions['Category'].value_counts()

#### CHECK FOR DUPLICATE ROWS

In [ ]:
Total_dup = customer_transactions.duplicated().sum()
print('The total number of duplicate rows in this dataset is', Total_dup)

In [ ]:
customer_transactions.describe().T

In [ ]:
customer_transactions.describe(include='object').T

### EXPLORATORY DATA ANALYSIS, (E.D.A)

#### MAKE A COPY OF THE ORIGINAL DATASET.

In [ ]:
customer_transactions_copy = customer_transactions.copy

In [ ]:
Cols_drop = ['Customer ID', 'Last Name']
customer_transactions = customer_transactions.drop(columns= Cols_drop)

In [ ]:
customer_transactions.boxplot()

#### INSIGHTS TO BE DERIVED FROM THIS ANALYSIS

#### QUESTIONS

 - Which category made the highest sales during the period?
 - Which age grouping patronized more of cosmetics, clothings and travel?
 - Which date had the highest patronage for Clothing?

#### Question 1
 - Which category made the highest sales during the period?

In [ ]:
# Group the data by category and calculate total sales for each category
category_sales = customer_transactions.groupby('Category')['Transaction Amt ($)'].sum()

# Print the total sales for each category
print(category_sales)

In [ ]:
# Plot a bar chart to visualize the results
plt.figure(figsize=(10, 6))
category_sales.plot(kind='bar')
plt.title('Total Sales by Category')
plt.xlabel('Category')
plt.ylabel('Transaction Amt ($)')
plt.xticks(rotation=45)

plt.legend()

# Add labels on top of the bars
for i, v in enumerate(category_sales):
    plt.text(i, v + 200, f'{v:.2f}', ha='center', va='bottom', fontsize=10)

plt.show()

In [ ]:
# Find the category with the highest total sales
highest_sales_category = category_sales.idxmax()
highest_sales_amount = category_sales.max()

print(f"The category with the highest sales is '{highest_sales_category}' with a total sales amount of $ {highest_sales_amount:.2f}")

#### Question 2
 - Which age group patronized more of cosmetics, clothings and travel?

In [ ]:
# Convert the 'Date of Birth' column to datetime
customer_transactions['Date of Birth'] = pd.to_datetime(customer_transactions['Date of Birth'])

# Calculate age based on the current date
current_date = datetime.now()
customer_transactions['Age'] = (current_date - customer_transactions['Date of Birth']) // pd.Timedelta(days=365)

customer_transactions

In [ ]:
# Group Ages into categories: young, middle age, and old

# Define age bins and labels
age_bins = [0, 30, 50, 100]  # You can adjust these age boundaries as needed

age_labels = ['Young', 'Middle Age', 'Old']

# Apply age grouping
customer_transactions['Age Group'] = pd.cut(customer_transactions['Age'], bins=age_bins, labels=age_labels, right=False)


In [ ]:
customer_transactions.head()

In [ ]:
# Filter the data for the specified products
products_of_interest = ['Cosmetic', 'Clothing', 'Travel']  # Use correct case for column names
filtered_data = customer_transactions[customer_transactions['Category'].isin(products_of_interest)]

# Group the data by age group and count the number of purchases in each category
grouped_data = filtered_data.groupby(['Age Group', 'Category']).size().unstack().fillna(0)

# Find the age group that patronized each category the most
most_patronized_travel = grouped_data['Travel'].idxmax()
most_patronized_clothing = grouped_data['Clothing'].idxmax()
most_patronized_cosmetics = grouped_data['Cosmetic'].idxmax()

print(f"The age group that patronized travel the most is {most_patronized_travel}")
print(f"The age group that patronized clothing the most is {most_patronized_clothing}")
print(f"The age group that patronized cosmetics the most is {most_patronized_cosmetics}")

In [ ]:
# Create a pie chart to visualize the results
labels = ['Cosmetic', 'Clothing', 'Travel']
sizes = [grouped_data['Travel'].max(), grouped_data['Clothing'].max(), grouped_data['Cosmetic'].max()]
explode = (0.1, 0, 0)  # Explode the 1st slice (Cosmetic)

plt.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.title('Age Groups Patronizing Products')

plt.show()

#### Question 3
 - Which date had the highest patronage for Clothing?

In [ ]:
# List of categories you want to analyze
categories_to_analyze = ['Cosmetic', 'Clothing', 'Travel' , 'Electronics' , 'Market', 'Restaurant']

# Iterate through the categories
for category in categories_to_analyze:
    # Filter the data for the specific category
    category_data = customer_transactions[customer_transactions['Category'] == category]
    
    # Group the data by date and count the number of purchases for each date
    date_patronage = category_data.groupby('Date').size()
    
    # Find the date with the highest patronage for the category
    date_with_highest_patronage = date_patronage.idxmax()
    highest_patronage_count = date_patronage.max()
    
    print(f"For the '{category}' category:")
    print(f"The date with the highest patronage is {date_with_highest_patronage} with {highest_patronage_count} purchases.")
    print()


 #### Question 4
 - What is the total number of person based on their gender patronized restaurant?

In [ ]:
# Filter the data for the "Restaurant" category
restaurant_data = customer_transactions[customer_transactions['Category'] == 'Restaurant']

# Group the data by gender and count the number of patrons for each gender
gender_patrons = restaurant_data['Gender'].value_counts()

print("Total number of people who patronized the restaurant based on gender:")
print(gender_patrons)


In [ ]:
# Create a bar chart to visualize the results
gender_labels = gender_patrons.index
patron_counts = gender_patrons.values

plt.figure(figsize=(8, 6))
plt.bar(gender_labels, patron_counts, color='blue')
plt.title('Total Number of Restaurant Patrons by Gender')
plt.xlabel('Gender')
plt.ylabel('Number of Patrons')

# Annotate the bars with counts
for i, count in enumerate(patron_counts):
    plt.text(i, count, str(count), ha='center', va='bottom', fontsize=12)

plt.show()

 #### Question 5
 - What is the trend in the transactions?

In [ ]:
# Convert the 'transaction_date' column to a datetime object
customer_transactions['Date'] = pd.to_datetime(customer_transactions['Date'])

# Group the data by transaction date and count the number of transactions for each date
transaction_trend = customer_transactions.groupby('Date').size()

# Create a line chart to visualize the transaction trend
plt.figure(figsize=(12, 6))
plt.plot(transaction_trend.index, transaction_trend.values, marker='o', color='red')
plt.title('Transaction Trend Over The Transaction Period.')
plt.xlabel('Transaction Date')
plt.ylabel('Number of Transactions')
plt.grid(True)

# Optionally, format the x-axis labels for date
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.DayLocator(interval=10))  # Adjust the interval as needed

plt.xticks(rotation=45)

plt.show()

In [ ]:
# Convert the 'transaction_date' column to a datetime object
customer_transactions['Date'] = pd.to_datetime(customer_transactions['Date'])

# Group the data by transaction date and count the number of transactions for each date
transaction_counts_by_date = customer_transactions.groupby('Date').size()
date_with_lowest_transactions = transaction_counts_by_date.idxmin()

# Group the data by category and count the number of patrons for each category
category_patronage = customer_transactions['Category'].value_counts()
category_with_lowest_patronage = category_patronage.idxmin()

print(f"The date with the lowest number of transactions is {date_with_lowest_transactions}")
print(f"The category with the lowest patronage is {category_with_lowest_patronage}")


 #### Question 6
 - What is the percentage for the various gender? 

In [ ]:
#value counts for each gender

customer_transactions['Gender'].value_counts()

In [ ]:
# Assuming your DataFrame has a 'Gender' column, you can count the number of transactions for each gender
gender_counts = customer_transactions['Gender'].value_counts()

# Create a pie chart
plt.figure(figsize=(7, 7))
plt.pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Gender Distribution Percentage')

# Add a legend
plt.legend()

plt.show()

In [ ]:
# Group by "Age Group" and count the number of each gender within each age group
age_gender_distribution = customer_transactions.groupby(['Age Group', 'Gender']).size().unstack(fill_value=0)

# Display the age-gender distribution
print(age_gender_distribution)


In [ ]:
# Plot the age-gender distribution as a bar chart
age_gender_distribution.plot(kind='bar')
plt.title('Age-Gender Distribution')
plt.xlabel('Age Group')
plt.ylabel('Count')

plt.show()

 #### Question 7
 - Which merchant name had the highest number of transactions during the period?

In [ ]:
# Group the data by the merchant and count the number of transactions for each merchant

merchant_transaction_counts = customer_transactions['Merchant Name'].value_counts()

merchant_with_highest_transactions = merchant_transaction_counts.idxmax()

highest_transaction_count = merchant_transaction_counts.max()

print(f"The merchant with the highest number of transactions is '{merchant_with_highest_transactions}'")
print()
print(f"They have {highest_transaction_count} transactions.")